In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('../../SnowEx-Data/snowex_depths.csv', index_col=0)
gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df["geometry"]), crs="epsg:4326",)

In [ ]:
data_dir = Path('../../SnowEx-Data')
spicy_depths = []
# tolerance around each site 200 m
tol = 0.00090009*2

for fp in data_dir.glob('*.nc'):
    ds = xr.open_dataset(fp)
    name = fp.stem.replace('.sub','')
    xmin, ymin, xmax, ymax = ds.rio.bounds()
    site_depths = gdf.cx[xmin:xmax, ymin:ymax].copy()
    for i, r in site_depths.iterrows():
        depth_dt = pd.to_datetime(r.date)
        ds_ts = ds['snow_depth'].sel(time = slice(depth_dt - pd.Timedelta('1 day'), depth_dt + pd.Timedelta('1 day'))).mean(dim = 'time')
        spicy_d = ds_ts.sel(x = slice(r.geometry.x-tol, r.geometry.x + tol), y = slice(r.geometry.y + tol, r.geometry.y - tol)) #, method = 'nearest'
        site_depths.loc[i, 'spicy_depth'] = spicy_d.mean(skipna = True)*100
    spicy_depths.append(site_depths)

In [ ]:
df = pd.concat(spicy_depths, ignore_index=True)
df = df.dropna()
df = df[df.spicy_depth != 0]

In [ ]:
df.date.unique()

In [ ]:
fig, ax = plt.subplots()
cmap = {'Boise River':'b', 'Fraser':'r'}
 
for k, d in df.groupby('Location'):
    ax.scatter(d['depth'], d['spicy_depth'], label=k, c = cmap[k])
r, p = pearsonr(df.depth, df.spicy_depth)
rmse = mean_squared_error(df.depth, df.spicy_depth, squared=False)
ax.text(.01, .99, f'r: {r:.2}, rmse: {rmse:.2f}\nn = {len(df.spicy_depth):.2e}', ha = 'left', va = 'top', transform = ax.transAxes)
plt.xlabel('Probed Snow Depth')
plt.ylabel('Spicy Depth (200m box)')
plt.title('SnowEx Probed vs Spicy Depths')
plt.legend(loc = 'lower right')
plt.savefig('../../images/probed-spicy-plots/probed_scatter.png')